## データセット作成

In [ ]:
# ライブラリのインポート
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# データのダウンロード
train = pd.read_parquet('')
test = pd.read_parquet('')

# 関数群


In [ ]:
def reduce_mem_usage(df):
    """ メモリ削減 する関数
    
    Args:
        df(DataFrame) : データベース

    Return:
        df(DataFrame) : 加工したデータベース

    """

    # float64 と float32 のデータタイプを float16 に加工
    cols1 = list(df.dtypes[df.dtypes == 'float64'].index) + list(df.dtypes[df.dtypes == 'float32'].index) 
    for col in tqdm(cols1):
        df[col] = df[col].astype(np.float16)

    # int64, int32, int16 のデータタイプを int8 に加工 
    cols2 = list(df.dtypes[df.dtypes == 'int64'].index) + list(df.dtypes[df.dtypes == 'int32'].index)+ list(df.dtypes[df.dtypes == 'int16'].index)
    for col in tqdm(cols2):
        df[col] = df[col].astype(np.int8)

    return df

In [ ]:
def get_concat(df, df1):
    """ マージ する関数
    
    Args:
        df(DataFrame) : データベース
        df1(DataFrame) : 追加するデータベース

    Return:
        df(DataFrame) : マージしたデータベース

    """

    # マージする
    df = pd.concat([
        df.reset_index(drop=True),
        df1.reindex(df["customer_ID"].values).reset_index(drop=True)
    ], axis=1)

    # 不要なデータベースの削除
    del df1
    gc.collect()

    print("\n", f"Now {len(df.columns.tolist())} features...")

    # 余分なメモリの削除
    df = reduce_mem_usage(df)
    gc.collect() 

    return df

In [ ]:
def get_datetime(df):
    """ datetime に関するデータ加工の関数
    
    Args:
        df(DataFrame) : データベース

    Return:
        df(DataFrame) : 加工したデータベース

    """

    print("\n", "#"*4, "get datetime", "#"*4)
    # 処理の開始
    df1 = df.drop(df.columns.tolist(), axis=1).copy()
    df['S_2'] = pd.to_datetime(df['S_2'])
    df1['S_2_year'] = df['S_2'].dt.year
    df1['S_2_month'] = df['S_2'].dt.month
    df1['S_2_day'] = df['S_2'].dt.day
    df1['S_2_day_of_week'] = df['S_2'].dt.day_of_week
    df1['S_2_day_of_year'] = df['S_2'].dt.day_of_year
    df1['S_2_is_year_start'] = df['S_2'].dt.is_year_start
    df1['S_2_is_quarter_start'] = df['S_2'].dt.is_quarter_start
    df1['S_2_is_month_start'] = df['S_2'].dt.is_month_start
    df1['S_2_is_month_end'] = df['S_2'].dt.is_month_end
    df1['S_2_is_weekend'] = np.where(df1['S_2_day_of_week'].isin([5,6]), 1,0)
    df1['month_sin'] = np.sin(2 * np.pi * df['S_2'].dt.month/12.0) 
    df1['month_cos'] = np.cos(2 * np.pi * df['S_2'].dt.month/12.0)
    
    # 既存データベースにマージする
    df = get_concat(df, df1)
    
    # 日付の変数はもう要らないので削除
    df = df.drop("S_2", axis = 1)
    
    return df

In [ ]:
def get_shift(df, num_features, groups):
    """ シフト変数を追加する関数
    
    Args:
        df(DataFrame) : データベース
        num_features(list) : 数値型の変数リスト
        groups(DataFrame) : customer_ID ごとにグループバイしたデータベース

    Return:
        df(DataFrame) : 加工したデータベース

    """

    print("\n", "#"*4, "get shift", "#"*4)
    df1 = []
    customer_ids = []
    # customer_ID ごとにデータを作成
    for customer_id, df in tqdm(groups):
        diff_df1 = df[num_features].shift(1).iloc[[-1]].values.astype(np.float32) 
        df1.append(diff_df1)  
        customer_ids.append(customer_id) 

    # データベースとして作成
    df1 = np.concatenate(df1, axis = 0)
    df1 = pd.DataFrame(df1, columns = [col + '_shift' for col in df[num_features].columns])
    df1['customer_ID'] = customer_ids
    
    # 既存のデータベースとマージ
    df = get_concat(df, df1)

    return df

In [ ]:
def get_diff(df, num_features, groups):
    """ 差分を追加する関数
    
    Args:
        df(DataFrame) : データベース
        num_features(list) : 数値型の変数リスト
        groups(DataFrame) : customer_ID ごとにグループバイしたデータベース

    Return:
        df(DataFrame) : 加工したデータベース

    """

    print("\n", "#"*4, "get difference", "#"*4)
    df1 = []
    customer_ids = []
    # customer_ID ごとにデータを作成
    for customer_id, df in tqdm(groups):
        diff_df1 = df[num_features].diff().iloc[[-1]].values.astype(np.float32) 
        df1.append(diff_df1)  
        customer_ids.append(customer_id) 

    # データベースとして作成
    df1 = np.concatenate(df1, axis = 0)
    df1 = pd.DataFrame(df1, columns = [col + '_diff' for col in df[num_features].columns])
    df1['customer_ID'] = customer_ids
    
    # 既存のデータベースとマージ
    df = get_concat(df, df1)
    
    return df

In [ ]:
def get_mean(df, num_features, groups):
    """ 平均を追加する関数
    
    Args:
        df(DataFrame) : データベース
        num_features(list) : 数値型の変数リスト
        groups(DataFrame) : customer_ID ごとにグループバイしたデータベース

    Return:
        df(DataFrame) : 加工したデータベース

    """

    print("\n", "#"*4, "get mean", "#"*4)
    # customer_ID ごとの mean を作成
    df1 = groups[num_features].agg(['mean']).copy()
    df1.columns = [col + '_mean' for col in num_features]

    # 既存のデータベースとマージ
    df = get_concat(df, df1)
    
    return df

In [ ]:
def get_std(df, num_features, groups):
    """ 標準偏差を追加する関数
    
    Args:
        df(DataFrame) : データベース
        num_features(list) : 数値型の変数リスト
        groups(DataFrame) : customer_ID ごとにグループバイしたデータベース

    Return:
        df(DataFrame) : 加工したデータベース

    """

    print("\n", "#"*4, "get std", "#"*4)
    # customer_ID ごとの std を作成
    df1 = groups[num_features].agg(['std']).copy()
    df1.columns = [col + '_std' for col in num_features]

    # 既存のデータベースとマージ
    df = get_concat(df, df1)
    
    return df

In [ ]:
def get_median(df, num_features, groups):
    """ 中央値を追加する関数
    
    Args:
        df(DataFrame) : データベース
        num_features(list) : 数値型の変数リスト
        groups(DataFrame) : customer_ID ごとにグループバイしたデータベース

    Return:
        df(DataFrame) : 加工したデータベース

    """

    print("\n", "#"*4, "get median", "#"*4)
    # customer_ID ごとの median を作成
    df1 = groups[num_features].agg(['median']).copy()
    df1.columns = [col + '_median' for col in num_features]
    
    # 既存のデータベースとマージ
    df = get_concat(df, df1)
    
    return df

In [ ]:
def get_min(df, num_features, groups):
    """ 最小値を追加する関数
    
    Args:
        df(DataFrame) : データベース
        num_features(list) : 数値型の変数リスト
        groups(DataFrame) : customer_ID ごとにグループバイしたデータベース

    Return:
        df(DataFrame) : 加工したデータベース

    """

    print("\n", "#"*4, "get min", "#"*4)
    # customer_ID ごとの min を作成
    df1 = groups[num_features].agg(['min']).copy()
    df1.columns = [col + '_min' for col in num_features]

    # 既存のデータベースとマージ
    df = get_concat(df, df1)
        
    return df

In [ ]:
def get_max(df, num_features, groups):
    """ 最大値を追加する関数
    
    Args:
        df(DataFrame) : データベース
        num_features(list) : 数値型の変数リスト
        groups(DataFrame) : customer_ID ごとにグループバイしたデータベース

    Return:
        df(DataFrame) : 加工したデータベース

    """

    print("\n", "#"*4, "get max", "#"*4)
    # customer_ID ごとの max を作成
    df1 = groups[num_features].agg(['max']).copy()
    df1.columns = [col + '_max' for col in num_features]

    # 既存のデータベースとマージ
    df = get_concat(df, df1)
        
    return df

In [ ]:
def get_var(df, num_features, groups):
    """ 分散を追加する関数
    
    Args:
        df(DataFrame) : データベース
        num_features(list) : 数値型の変数リスト
        groups(DataFrame) : customer_ID ごとにグループバイしたデータベース

    Return:
        df(DataFrame) : 加工したデータベース

    """
    
    print("\n", "#"*4, "get var", "#"*4)
    # customer_ID ごとの var を作成
    df1 = groups[num_features].agg(['var']).copy()
    df1.columns = [col + '_var' for col in num_features]

    # 既存のデータベースとマージ
    df = get_concat(df, df1)
        
    return df

In [ ]:
def get_first(df, features, groups):
    """ 最初の値を追加する関数
    
    Args:
        df(DataFrame) : データベース
        features(list) : 変数名リスト
        groups(DataFrame) : customer_ID ごとにグループバイしたデータベース

    Return:
        df(DataFrame) : 加工したデータベース

    """
    
    print("\n", "#"*4, "get first", "#"*4)
    # customer_ID ごとの first を作成
    df1 = groups[features].agg(['first']).copy()
    df1.columns = [col + '_first' for col in features]

    # 既存のデータベースとマージ
    df = get_concat(df, df1)
            
    return df

In [ ]:
def get_last(df, features, groups):
    """ 最初の値を追加する関数
    
    Args:
        df(DataFrame) : データベース
        features(list) : 変数名リスト
        groups(DataFrame) : customer_ID ごとにグループバイしたデータベース

    Return:
        df(DataFrame) : 加工したデータベース

    """

    print("\n", "#"*4, "get last", "#"*4)
    # customer_ID ごとの last を作成
    df1 = groups[features].agg(['last']).copy()
    df1.columns = [col + '_last' for col in features]

    # 既存のデータベースとマージ
    df = get_concat(df, df1)
    
    return df

In [ ]:
def get_count(df, cat_features, groups):
    """ 頻度を追加する関数
    
    Args:
        df(DataFrame) : データベース
        cat_features(list) : カテゴリ型の変数名リスト
        groups(DataFrame) : customer_ID ごとにグループバイしたデータベース

    Return:
        df(DataFrame) : 加工したデータベース

    """

    print("\n", "#"*4, "get count", "#"*4)
    # customer_ID ごとの count を作成
    df1 = groups[cat_features].agg(['count']).copy()
    df1.columns = [col + '_count' for col in cat_features]

    # 既存のデータベースとマージ
    df = get_concat(df, df1)

    return df

In [ ]:
def get_nunique(df, cat_features, groups):
    """ 種数(一意の値の個数)を追加する関数
    
    Args:
        df(DataFrame) : データベース
        cat_features(list) : カテゴリ型の変数名リスト
        groups(DataFrame) : customer_ID ごとにグループバイしたデータベース

    Return:
        df(DataFrame) : 加工したデータベース

    """

    print("\n", "#"*4, "get nunique", "#"*4)
    # customer_ID ごとの nunique を作成
    df1 = groups[cat_features].agg(['nunique']).copy()
    df1.columns = [col + '_nunique' for col in cat_features]

    # 既存のデータベースとマージ
    df = get_concat(df, df1)
    
    return df

In [ ]:
def get_current_level(df, num_features, df_drop):
    """ 現在の水準レベルを追加する関数
    
    Args:
        df(DataFrame) : データベース
        num_features(list) : 数値型の変数名リスト

    Return:
        df(DataFrame) : 加工したデータベース

    """

    print("\n", "#"*4, "get current lebel", "#"*4)
    # 各列 ごとの current_level を作成
    new_columns = {f'{col}_current_level': (df[f"{col}_last"] - df[f"{col}_min"]) / (df[f"{col}_max"] - df[f"{col}_min"]) for col in tqdm(num_features)}
    new_df = pd.DataFrame(new_columns)
    df1 = pd.concat([df_drop, new_df], axis=1)

    del new_df
    gc.collect()

    # 既存のデータベースとマージ
    df = get_concat(df, df1)
    
    return df

In [ ]:
def get_magnitude(df, num_features, df_drop):
    """ 振れ幅を追加する関数
    
    Args:
        df(DataFrame) : データベース
        num_features(list) : 数値型の変数名リスト

    Return:
        df(DataFrame) : 加工したデータベース

    """

    print("\n", "#"*4, "get magnitude", "#"*4)
    # 各列 ごとの magnitude を作成
    new_columns = {f'{col}_magnitude': df[f"{col}_max"] - df[f"{col}_min"] for col in tqdm(num_features)}
    new_df = pd.DataFrame(new_columns)
    df1 = pd.concat([df_drop, new_df], axis=1)

    del new_df
    gc.collect()

    # 既存のデータベースとマージ
    df = get_concat(df, df1)

    return df

In [ ]:
def get_col_last_diff(df, num_features, df_drop):
    """ 最近の値との差分を追加する関数
    
    Args:
        df(DataFrame) : データベース
        num_features(list) : 数値型の変数名リスト

    Return:
        df(DataFrame) : 加工したデータベース

    """

    print("\n", "#"*4, "get col_last diff", "#"*4)
    # 各列 ごとの col_last_diff を作成
    new_columns = {f'{col}_lag_sub' : (df[col] - df[col.replace('last', col_2)]) for col in tqdm(num_features) for col_2 in ['first', 'mean', 'std', 'min', 'max'] if 'last' in col  and col.replace('last', col_2) in df}
    new_columns2 = {f'{col}_lag_div' : df[col] / df[col.replace('last', col_2)] for col in tqdm(num_features) for col_2 in ['first', 'mean', 'std', 'min', 'max'] if 'last' in col and col.replace('last', col_2) in df}
    new_columns.update(new_columns2)
    new_df = pd.DataFrame(new_columns)
    df1 = pd.concat([df_drop, new_df], axis=1)    

    del new_df
    gc.collect()

    # 既存のデータベースとマージ
    df = get_concat(df, df1)
    
    return df

In [ ]:
def get_after_pay(df, df_drop):
    """ after_pay を追加する関数
    
    Args:
        df(DataFrame) : データベース

    Return:
        df(DataFrame) : 加工したデータベース

    """

    print("\n", "#"*4, "get after pay", "#"*4)
    # 各列 ごとの after_pay を作成
    new_columns = {f'{bcol}-{pcol}' : df[bcol] - df[pcol] for bcol in tqdm(['B_11','B_14','B_17','D_39','D_131','S_16','S_23']) for pcol in ['P_2','P_3'] if bcol in df.columns}
    new_df = pd.DataFrame(new_columns)
    df1 = pd.concat([df_drop, new_df], axis=1)    
    
    del new_df
    gc.collect()

    # 既存のデータベースとマージ
    df = get_concat(df, df1)
    
    return df

In [ ]:
from sklearn.preprocessing import LabelEncoder
def get_label(df, cat_features, df_drop):
    """ label encode する関数
    
    Args:
        df(DataFrame) : データベース
        cat_features(list) : カテゴリ変数名

    Return:
        df(DataFrame) : 加工したデータベース

    """ 

    print("\n", "#"*4, "get label", "#"*4)
    # カテゴリ変数毎にラベルエンコーディング
    encoder = LabelEncoder()
    new_columns = {f"{cat_col}_label" : encoder.fit_transform(df[cat_col]) for cat_col in tqdm(cat_features)}
    new_df = pd.DataFrame(new_columns)
    df1 = pd.concat([df_drop, new_df], axis=1)    
    
    del new_df
    gc.collect()

    # 既存のデータベースとマージ
    df = get_concat(df, df1)
        
    # 余分なメモリの削除
    df = reduce_mem_usage(df)
    gc.collect() 
    
    return df

In [ ]:
def get_frequency(df, cat_features, len_train, df_drop):
    """ frequency encode する関数
    
    Args:
        df(DataFrame) : データベース
        cat_features(list) : カテゴリ変数名
        len_train(int) : データベースの行数

    Return:
        df(DataFrame) : 加工したデータベース

    """
    
    print("\n", "#"*4, "get frequency", "#"*4)
    # カテゴリ変数毎にフリークエンシーエンコーディング
    new_columns = {f"{cat_col}_freq" : df[cat_col].map(dict(df[cat_col].value_counts())) / len_train for cat_col in tqdm(cat_features)}
    new_df = pd.DataFrame(new_columns)
    df1 = pd.concat([df_drop, new_df], axis=1)    
    
    del new_df
    gc.collect()
    
    # 既存のデータベースとマージ
    df = get_concat(df, df1)
        
    # 余分なメモリの削除
    df = reduce_mem_usage(df)
    gc.collect() 

    return df

In [ ]:
def mark_imputed(df, missing_cols, df_drop):
    """欠損値の有無をメモ、欠損値を -1 で補完する関数
    
    Args:
        df(DataFrame) : データベース
        missing_cols(list) : 既存データベースのうち、欠損値がある変数名

    Return:
        df(DataFrame) : 加工したデータベース

    """

    print("\n", "#"*4, "get imputation", "#"*4)
    # 変数毎に欠損値をマーク
    new_columns = {f"{col}__missing" : df[col].isna() for col in tqdm(missing_cols)}
    new_df = pd.DataFrame(new_columns)
    df1 = pd.concat([df_drop, new_df], axis=1)    
    
    del new_df
    gc.collect()

    # 既存のデータベースとマージ
    df = get_concat(df, df1)
    
    # 欠損値を -1 に補完
    for col in tqdm(df.columns.tolist()):
        df[col].fillna(-1)
        
    # 余分なメモリの削除
    df = reduce_mem_usage(df)
    gc.collect() 
    
    return df

# まとめ

In [ ]:
def preprocess_all(train):
    """用意した加工を行ってまとめる関数
    
    Args:
        train(DataFrame) : 既存のデータベース

    Return:
        new_df(DataFrame) : 加工した変数が全て入ったデータベース

    """
    
    # 余分なメモリの削除
    new_df = reduce_mem_usage(train)
    
    # リストの作成
    features = train.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68",
    ]    
    num_features = [col for col in features if col not in cat_features]
    missing_cols = train.drop(['S_2'], axis = 1).isna().columns.to_list()
    len_train = len(train)
    
    # グループバイデータベースの作成
    groups = train.groupby("customer_ID")
    
    # 空のデータベースの作成
    df_drop = train.drop(train.columns.tolist(), axis=1).copy()

    # 不要なデータベースの削除
    del train
    gc.collect()
    
    # 関数の実行
    new_df = get_datetime(new_df)
    new_df = get_shift(new_df, num_features, groups)
    new_df = get_diff(new_df, num_features, groups)
    new_df = get_mean(new_df, num_features, groups)
    new_df = get_std(new_df, num_features, groups)
    new_df = get_median(new_df, num_features, groups)
    new_df = get_min(new_df, num_features, groups)
    new_df = get_max(new_df, num_features, groups)
    new_df = get_var(new_df, num_features, groups)
    new_df = get_first(new_df, features, groups)
    new_df = get_last(new_df, features, groups)
    new_df = get_count(new_df, cat_features, groups)
    new_df = get_nunique(new_df, cat_features, groups)
    new_df = get_current_level(new_df, num_features, df_drop)
    new_df = get_magnitude(new_df, num_features, df_drop)
    new_df = get_col_last_diff(new_df, num_features, df_drop)
    new_df = get_after_pay(new_df, df_drop)
    new_df = get_label(new_df, cat_features, df_drop)
    new_df = get_frequency(new_df, cat_features, len_train, df_drop)
    new_df = mark_imputed(new_df, missing_cols, df_drop)
    
    return new_df

In [ ]:
new_train = preprocess_all(train)

In [ ]:
new_test = preprocess_all(test)